# Data Wrangling y Enriquecimiento - Encuesta Fundación Teletón

Este notebook realiza el procesamiento, limpieza y enriquecimiento de los datos de la encuesta de satisfacción de Fundación Teletón, basada en el modelo **SERVQUAL**.

## Dimensiones SERVQUAL en el Dataset
- **Tangibles (AT)**: Apariencia física, procedimientos y documentación
- **Fiabilidad (FI)**: Cumplimiento de promesas, conocimiento, información clara
- **Responsiveness (R)**: Rapidez, disposición a ayudar, flexibilidad
- **Empatía (E)**: Comprensión, dedicación de tiempo, atención personalizada

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Carga de Datos

In [ ]:
# Cargar datos originales
df = pd.read_csv('../data/teleton-non-excel.csv', encoding='utf-8-sig')

print(f"Dimensiones del dataset: {df.shape[0]} registros x {df.shape[1]} columnas")
print(f"\nColumnas:\n{df.columns.tolist()}")

In [ ]:
# Vista previa de los datos
df.head()

In [ ]:
# Información del dataset
df.info()

In [ ]:
# Valores nulos por columna
nulos = df.isnull().sum()
print("Valores nulos por columna:")
print(nulos[nulos > 0] if nulos.sum() > 0 else "No hay valores nulos")

## 2. Limpieza y Tipos de Datos

In [ ]:
# Crear copia para trabajar
df_clean = df.copy()

# Renombrar columnas para consistencia
df_clean = df_clean.rename(columns={
    'R_12': 'NPS',  # Variable de recomendación (Net Promoter Score)
    'Info': 'INFO',
    'Años': 'AÑOS'
})

print("Columnas renombradas:")
print(df_clean.columns.tolist())

In [ ]:
# Convertir timestamp a datetime
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], format='%m/%d/%Y %H:%M:%S')

print(f"Rango de fechas: {df_clean['timestamp'].min()} a {df_clean['timestamp'].max()}")

In [ ]:
# Definir variables por tipo
vars_servqual = ['AT_1', 'AT_2', 'FI_1', 'FI_2', 'FI_3', 'R_1', 'R_2', 'R_3', 'E_1', 'E_2', 'E_3', 'E_4']
vars_outcome_5 = ['C_1']  # Escala 1-5
vars_outcome_10 = ['D_1', 'NPS', 'INFO']  # Escala 1-10
vars_categoricas = ['Giro', 'Puesto', 'Estado']

print(f"Variables SERVQUAL (escala 1-5): {len(vars_servqual)} variables")
print(f"Variables Outcome escala 1-5: {vars_outcome_5}")
print(f"Variables Outcome escala 1-10: {vars_outcome_10}")
print(f"Variables Categóricas: {vars_categoricas}")

In [ ]:
# Convertir variables numéricas a tipos óptimos
# Variables SERVQUAL y C_1 (escala 1-5) a Int8 (permite NaN)
for col in vars_servqual + vars_outcome_5:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int8')

# Variables outcome escala 1-10 a Int8
for col in vars_outcome_10:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int8')

# Años como Int16 (puede haber valores > 127)
df_clean['AÑOS'] = pd.to_numeric(df_clean['AÑOS'], errors='coerce').astype('Int16')

print("Tipos de datos actualizados para variables numéricas")

In [ ]:
# Convertir variables categóricas
for col in vars_categoricas:
    df_clean[col] = df_clean[col].astype('category')

# Verificar categorías únicas
for col in vars_categoricas:
    print(f"\n{col}: {df_clean[col].nunique()} categorías")
    print(df_clean[col].value_counts())

In [ ]:
# Verificar tipos de datos finales
df_clean.info()

In [ ]:
# Estadísticas descriptivas de variables numéricas
df_clean[vars_servqual + vars_outcome_5 + vars_outcome_10 + ['AÑOS']].describe()

## 3. Variables Temporales Derivadas

In [ ]:
# Extraer componentes temporales
df_clean['fecha'] = df_clean['timestamp'].dt.date
df_clean['mes'] = df_clean['timestamp'].dt.month
df_clean['dia_semana'] = df_clean['timestamp'].dt.day_name()
df_clean['dia_semana_num'] = df_clean['timestamp'].dt.dayofweek
df_clean['hora'] = df_clean['timestamp'].dt.hour

# Crear variable de turno
def clasificar_turno(hora):
    if 6 <= hora < 12:
        return 'Mañana'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noche'

df_clean['turno'] = df_clean['hora'].apply(clasificar_turno).astype('category')

print("Variables temporales creadas:")
print(df_clean[['timestamp', 'fecha', 'mes', 'dia_semana', 'hora', 'turno']].head(10))

In [ ]:
# Distribución de respuestas por turno
print("Respuestas por turno:")
print(df_clean['turno'].value_counts())

In [ ]:
# Distribución por día de la semana
print("\nRespuestas por día de la semana:")
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(df_clean['dia_semana'].value_counts().reindex(dias_orden))

## 4. Scores SERVQUAL Agregados

In [ ]:
# Definir ítems por dimensión SERVQUAL
dimensiones_servqual = {
    'tangibles': ['AT_1', 'AT_2'],
    'fiabilidad': ['FI_1', 'FI_2', 'FI_3'],
    'responsiveness': ['R_1', 'R_2', 'R_3'],
    'empatia': ['E_1', 'E_2', 'E_3', 'E_4']
}

# Calcular score promedio por dimensión
for dimension, items in dimensiones_servqual.items():
    df_clean[f'score_{dimension}'] = df_clean[items].mean(axis=1).round(2)
    print(f"Score {dimension}: promedio de {items}")

# Score SERVQUAL total (promedio de las 4 dimensiones)
scores_dimensiones = [f'score_{d}' for d in dimensiones_servqual.keys()]
df_clean['score_servqual_total'] = df_clean[scores_dimensiones].mean(axis=1).round(2)

print(f"\nScore SERVQUAL total: promedio de {scores_dimensiones}")

In [ ]:
# Verificar scores calculados
df_clean[['score_tangibles', 'score_fiabilidad', 'score_responsiveness', 'score_empatia', 'score_servqual_total']].describe()

## 5. Variables Discretas / Categóricas Derivadas

In [ ]:
# Categoría NPS (Net Promoter Score)
# Detractores: 1-6, Pasivos: 7-8, Promotores: 9-10
def clasificar_nps(valor):
    if pd.isna(valor):
        return np.nan
    elif valor <= 6:
        return 'Detractor'
    elif valor <= 8:
        return 'Pasivo'
    else:
        return 'Promotor'

df_clean['nps_categoria'] = df_clean['NPS'].apply(clasificar_nps).astype('category')

print("Distribución NPS:")
print(df_clean['nps_categoria'].value_counts())

In [ ]:
# Calcular NPS Score oficial
# NPS = % Promotores - % Detractores
nps_counts = df_clean['nps_categoria'].value_counts(normalize=True) * 100
nps_score = nps_counts.get('Promotor', 0) - nps_counts.get('Detractor', 0)

print(f"\nNPS Score de Fundación Teletón: {nps_score:.1f}")
print(f"  - Promotores: {nps_counts.get('Promotor', 0):.1f}%")
print(f"  - Pasivos: {nps_counts.get('Pasivo', 0):.1f}%")
print(f"  - Detractores: {nps_counts.get('Detractor', 0):.1f}%")

In [ ]:
# Nivel de satisfacción (D_1)
# Bajo: 1-4, Medio: 5-7, Alto: 8-10
def clasificar_satisfaccion(valor):
    if pd.isna(valor):
        return np.nan
    elif valor <= 4:
        return 'Bajo'
    elif valor <= 7:
        return 'Medio'
    else:
        return 'Alto'

df_clean['satisfaccion_nivel'] = df_clean['D_1'].apply(clasificar_satisfaccion).astype('category')

print("Distribución de Satisfacción:")
print(df_clean['satisfaccion_nivel'].value_counts())

In [ ]:
# Nivel de calidad percibida (C_1)
# Deficiente: 1-2, Regular: 3, Bueno: 4-5
def clasificar_calidad(valor):
    if pd.isna(valor):
        return np.nan
    elif valor <= 2:
        return 'Deficiente'
    elif valor == 3:
        return 'Regular'
    else:
        return 'Bueno'

df_clean['calidad_nivel'] = df_clean['C_1'].apply(clasificar_calidad).astype('category')

print("Distribución de Calidad Percibida:")
print(df_clean['calidad_nivel'].value_counts())

In [ ]:
# Antigüedad como benefactor
# Nuevo: 1-2 años, Establecido: 3-5 años, Veterano: 6+ años
def clasificar_antiguedad(valor):
    if pd.isna(valor):
        return np.nan
    elif valor <= 2:
        return 'Nuevo'
    elif valor <= 5:
        return 'Establecido'
    else:
        return 'Veterano'

df_clean['antiguedad_grupo'] = df_clean['AÑOS'].apply(clasificar_antiguedad).astype('category')

print("Distribución por Antigüedad:")
print(df_clean['antiguedad_grupo'].value_counts())

In [ ]:
# Nivel de información percibida (INFO)
# Bajo: 1-4, Medio: 5-7, Alto: 8-10
def clasificar_info(valor):
    if pd.isna(valor):
        return np.nan
    elif valor <= 4:
        return 'Desinformado'
    elif valor <= 7:
        return 'Parcialmente informado'
    else:
        return 'Bien informado'

df_clean['info_nivel'] = df_clean['INFO'].apply(clasificar_info).astype('category')

print("Distribución de Nivel de Información:")
print(df_clean['info_nivel'].value_counts())

## 6. Enriquecimiento Geográfico

In [ ]:
# Mapeo de estados a regiones geográficas de México
regiones_mexico = {
    'Aguascalientes': 'Centro-Norte',
    'Baja California': 'Noroeste',
    'Baja California Sur': 'Noroeste',
    'Campeche': 'Sureste',
    'Chiapas': 'Sureste',
    'Chihuahua': 'Norte',
    'Ciudad de México': 'Centro',
    'Coahuila': 'Norte',
    'Colima': 'Occidente',
    'Durango': 'Norte',
    'Estado de México': 'Centro',
    'Guanajuato': 'Bajío',
    'Guerrero': 'Sur',
    'Hidalgo': 'Centro',
    'Jalisco': 'Occidente',
    'Michoacán': 'Occidente',
    'Morelos': 'Centro',
    'Nayarit': 'Occidente',
    'Nuevo León': 'Noreste',
    'Oaxaca': 'Sur',
    'Puebla': 'Centro',
    'Querétaro': 'Bajío',
    'Quintana Roo': 'Sureste',
    'San Luis Potosí': 'Centro-Norte',
    'Sinaloa': 'Noroeste',
    'Sonora': 'Noroeste',
    'Tabasco': 'Sureste',
    'Tamaulipas': 'Noreste',
    'Tlaxcala': 'Centro',
    'Veracruz': 'Golfo',
    'Yucatán': 'Sureste',
    'Zacatecas': 'Centro-Norte'
}

# Limpiar estados con valores múltiples (ej: "Ciudad de México, Morelos")
df_clean['Estado_limpio'] = df_clean['Estado'].astype(str).apply(
    lambda x: x.split(',')[0].strip() if ',' in x else x
)

# Asignar región
df_clean['region'] = df_clean['Estado_limpio'].map(regiones_mexico).astype('category')

print("Estados únicos en el dataset:")
print(df_clean['Estado_limpio'].value_counts())

In [ ]:
# Distribución por región
print("\nDistribución por Región:")
print(df_clean['region'].value_counts())

In [ ]:
# Datos de población por estado (INEGI 2020 - en millones)
poblacion_estados = {
    'Aguascalientes': 1.43,
    'Baja California': 3.77,
    'Baja California Sur': 0.80,
    'Campeche': 0.93,
    'Chiapas': 5.54,
    'Chihuahua': 3.74,
    'Ciudad de México': 9.21,
    'Coahuila': 3.15,
    'Colima': 0.73,
    'Durango': 1.83,
    'Estado de México': 16.99,
    'Guanajuato': 6.17,
    'Guerrero': 3.54,
    'Hidalgo': 3.08,
    'Jalisco': 8.35,
    'Michoacán': 4.75,
    'Morelos': 1.97,
    'Nayarit': 1.29,
    'Nuevo León': 5.78,
    'Oaxaca': 4.13,
    'Puebla': 6.58,
    'Querétaro': 2.37,
    'Quintana Roo': 1.86,
    'San Luis Potosí': 2.82,
    'Sinaloa': 3.03,
    'Sonora': 2.94,
    'Tabasco': 2.40,
    'Tamaulipas': 3.53,
    'Tlaxcala': 1.34,
    'Veracruz': 8.06,
    'Yucatán': 2.32,
    'Zacatecas': 1.62
}

# PIB per cápita por estado (miles de pesos, 2020)
pib_per_capita_estados = {
    'Aguascalientes': 198,
    'Baja California': 214,
    'Baja California Sur': 247,
    'Campeche': 442,
    'Chiapas': 58,
    'Chihuahua': 218,
    'Ciudad de México': 394,
    'Coahuila': 263,
    'Colima': 173,
    'Durango': 142,
    'Estado de México': 114,
    'Guanajuato': 157,
    'Guerrero': 79,
    'Hidalgo': 115,
    'Jalisco': 183,
    'Michoacán': 107,
    'Morelos': 108,
    'Nayarit': 109,
    'Nuevo León': 310,
    'Oaxaca': 73,
    'Puebla': 113,
    'Querétaro': 250,
    'Quintana Roo': 211,
    'San Luis Potosí': 161,
    'Sinaloa': 151,
    'Sonora': 219,
    'Tabasco': 180,
    'Tamaulipas': 182,
    'Tlaxcala': 80,
    'Veracruz': 118,
    'Yucatán': 149,
    'Zacatecas': 118
}

# Agregar datos al dataframe
df_clean['poblacion_millones'] = df_clean['Estado_limpio'].map(poblacion_estados)
df_clean['pib_per_capita_miles'] = df_clean['Estado_limpio'].map(pib_per_capita_estados)

print("Datos geográficos agregados:")
df_clean[['Estado_limpio', 'region', 'poblacion_millones', 'pib_per_capita_miles']].drop_duplicates().head(10)

In [ ]:
# Calcular respuestas por millón de habitantes por estado
respuestas_por_estado = df_clean.groupby('Estado_limpio').size().reset_index(name='n_respuestas')
respuestas_por_estado['poblacion'] = respuestas_por_estado['Estado_limpio'].map(poblacion_estados)
respuestas_por_estado['respuestas_por_millon'] = (respuestas_por_estado['n_respuestas'] / respuestas_por_estado['poblacion']).round(2)

print("Densidad de respuestas por estado (por millón de habitantes):")
print(respuestas_por_estado.sort_values('respuestas_por_millon', ascending=False))

In [ ]:
# Clasificar nivel económico del estado
def clasificar_nivel_economico(pib):
    if pd.isna(pib):
        return np.nan
    elif pib < 120:
        return 'Bajo'
    elif pib < 200:
        return 'Medio'
    else:
        return 'Alto'

df_clean['nivel_economico_estado'] = df_clean['pib_per_capita_miles'].apply(clasificar_nivel_economico).astype('category')

print("\nDistribución por nivel económico del estado:")
print(df_clean['nivel_economico_estado'].value_counts())

In [ ]:
# Coordenadas de centroides de estados de México (lat, long)
coordenadas_estados = {
    'Aguascalientes': (21.8853, -102.2916),
    'Baja California': (30.8406, -115.2838),
    'Baja California Sur': (26.0444, -111.6661),
    'Campeche': (19.8301, -90.5349),
    'Chiapas': (16.7569, -93.1292),
    'Chihuahua': (28.6330, -106.0691),
    'Ciudad de México': (19.4326, -99.1332),
    'Coahuila': (27.0587, -101.7068),
    'Colima': (19.2452, -103.7241),
    'Durango': (24.0277, -104.6532),
    'Estado de México': (19.4969, -99.7233),
    'Guanajuato': (21.0190, -101.2574),
    'Guerrero': (17.4392, -99.5451),
    'Hidalgo': (20.0911, -98.7624),
    'Jalisco': (20.6595, -103.3494),
    'Michoacán': (19.5665, -101.7068),
    'Morelos': (18.6813, -99.1013),
    'Nayarit': (21.7514, -104.8455),
    'Nuevo León': (25.5922, -99.9962),
    'Oaxaca': (17.0732, -96.7266),
    'Puebla': (19.0414, -98.2063),
    'Querétaro': (20.5888, -100.3899),
    'Quintana Roo': (19.1817, -88.4791),
    'San Luis Potosí': (22.1565, -100.9855),
    'Sinaloa': (24.8091, -107.3940),
    'Sonora': (29.2972, -110.3309),
    'Tabasco': (17.8409, -92.6189),
    'Tamaulipas': (24.2669, -98.8363),
    'Tlaxcala': (19.3139, -98.2404),
    'Veracruz': (19.1738, -96.1342),
    'Yucatán': (20.7099, -89.0943),
    'Zacatecas': (22.7709, -102.5832)
}

# Agregar lat y long al dataframe
df_clean['lat'] = df_clean['Estado_limpio'].map(lambda x: coordenadas_estados.get(x, (None, None))[0])
df_clean['long'] = df_clean['Estado_limpio'].map(lambda x: coordenadas_estados.get(x, (None, None))[1])

print("Coordenadas agregadas:")
df_clean[['Estado_limpio', 'lat', 'long']].drop_duplicates().head(10)

## 7. Resumen del Dataset Enriquecido

In [ ]:
# Ver todas las columnas del dataset enriquecido
print(f"Dataset enriquecido: {df_clean.shape[0]} registros x {df_clean.shape[1]} columnas")
print(f"\nColumnas originales: {df.shape[1]}")
print(f"Columnas nuevas: {df_clean.shape[1] - df.shape[1]}")

print("\n--- COLUMNAS DEL DATASET ENRIQUECIDO ---")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col} ({df_clean[col].dtype})")

In [ ]:
# Vista previa del dataset final
df_clean.head()

In [ ]:
# Información del dataset final
df_clean.info()

In [ ]:
# Resumen de variables categóricas derivadas
vars_categoricas_nuevas = ['nps_categoria', 'satisfaccion_nivel', 'calidad_nivel', 'antiguedad_grupo', 'info_nivel', 'turno', 'region', 'nivel_economico_estado']

print("=" * 60)
print("RESUMEN DE VARIABLES CATEGÓRICAS DERIVADAS")
print("=" * 60)

for var in vars_categoricas_nuevas:
    print(f"\n>>> {var}")
    print(df_clean[var].value_counts())

## 8. Exportar Dataset Enriquecido

In [ ]:
# Ordenar columnas de manera lógica
columnas_ordenadas = [
    # Identificadores temporales
    'timestamp', 'fecha', 'mes', 'dia_semana', 'dia_semana_num', 'hora', 'turno',
    # Variables SERVQUAL originales
    'AT_1', 'AT_2', 'FI_1', 'FI_2', 'FI_3', 'R_1', 'R_2', 'R_3', 'E_1', 'E_2', 'E_3', 'E_4',
    # Scores SERVQUAL
    'score_tangibles', 'score_fiabilidad', 'score_responsiveness', 'score_empatia', 'score_servqual_total',
    # Variables outcome
    'D_1', 'satisfaccion_nivel', 'NPS', 'nps_categoria', 'C_1', 'calidad_nivel', 'INFO', 'info_nivel',
    # Variables demográficas
    'AÑOS', 'antiguedad_grupo', 'Giro', 'Puesto',
    # Variables geográficas
    'Estado', 'Estado_limpio', 'region', 'poblacion_millones', 'pib_per_capita_miles', 'nivel_economico_estado',
    # Coordenadas geográficas
    'lat', 'long'
]

df_final = df_clean[columnas_ordenadas]

print(f"Dataset final: {df_final.shape[0]} registros x {df_final.shape[1]} columnas")

In [ ]:
# Exportar a CSV
df_final.to_csv('../data/teleton_enriched.csv', index=False, encoding='utf-8-sig')
print("Dataset exportado a: ../data/teleton_enriched.csv")

# Exportar a Parquet (más eficiente para análisis)
df_final.to_parquet('../data/teleton_enriched.parquet', index=False)
print("Dataset exportado a: ../data/teleton_enriched.parquet")

## 9. Diccionario de Variables del Dataset Enriquecido

In [ ]:
diccionario_variables = """
# Diccionario de Variables - Dataset Enriquecido Teletón

## Variables Temporales
| Variable | Descripción | Tipo |
|----------|-------------|------|
| timestamp | Fecha y hora de la respuesta | datetime |
| fecha | Fecha de la respuesta | date |
| mes | Mes de la respuesta (1-12) | int |
| dia_semana | Nombre del día de la semana | category |
| dia_semana_num | Número del día (0=Lunes, 6=Domingo) | int |
| hora | Hora de la respuesta (0-23) | int |
| turno | Turno del día (Mañana/Tarde/Noche) | category |

## Variables SERVQUAL (escala 1-5)
| Variable | Dimensión | Descripción |
|----------|-----------|-------------|
| AT_1 | Tangibles | Se viste y comporta apropiadamente |
| AT_2 | Tangibles | Procedimientos y documentación claros |
| FI_1 | Fiabilidad | Cumple con horarios y plazos |
| FI_2 | Fiabilidad | Alto nivel de conocimiento |
| FI_3 | Fiabilidad | Información correcta y clara |
| R_1 | Responsiveness | Responde de forma rápida |
| R_2 | Responsiveness | Disposición a ayudar |
| R_3 | Responsiveness | Flexibilidad |
| E_1 | Empatía | Actitud comprensiva |
| E_2 | Empatía | Dedicación de tiempo |
| E_3 | Empatía | Entiende y se preocupa |
| E_4 | Empatía | Atención personalizada |

## Scores SERVQUAL Agregados
| Variable | Descripción | Rango |
|----------|-------------|-------|
| score_tangibles | Promedio de AT_1, AT_2 | 1-5 |
| score_fiabilidad | Promedio de FI_1, FI_2, FI_3 | 1-5 |
| score_responsiveness | Promedio de R_1, R_2, R_3 | 1-5 |
| score_empatia | Promedio de E_1, E_2, E_3, E_4 | 1-5 |
| score_servqual_total | Promedio de las 4 dimensiones | 1-5 |

## Variables Outcome
| Variable | Descripción | Escala |
|----------|-------------|--------|
| D_1 | Nivel de satisfacción | 1-10 |
| satisfaccion_nivel | Categoría de satisfacción | Bajo/Medio/Alto |
| NPS | Recomendación (Net Promoter) | 1-10 |
| nps_categoria | Categoría NPS | Detractor/Pasivo/Promotor |
| C_1 | Calidad percibida | 1-5 |
| calidad_nivel | Categoría de calidad | Deficiente/Regular/Bueno |
| INFO | Nivel de información | 1-10 |
| info_nivel | Categoría de información | Desinformado/Parcialmente/Bien informado |

## Variables Demográficas
| Variable | Descripción | Tipo |
|----------|-------------|------|
| AÑOS | Años como benefactor | int |
| antiguedad_grupo | Categoría de antigüedad | Nuevo/Establecido/Veterano |
| Giro | Tipo de organización | category |
| Puesto | Puesto del encuestado | category |

## Variables Geográficas
| Variable | Descripción | Tipo |
|----------|-------------|------|
| Estado | Estado original | category |
| Estado_limpio | Estado limpio (sin valores múltiples) | string |
| region | Región geográfica de México | category |
| poblacion_millones | Población del estado (millones) | float |
| pib_per_capita_miles | PIB per cápita (miles de pesos) | float |
| nivel_economico_estado | Nivel económico del estado | Bajo/Medio/Alto |
"""

print(diccionario_variables)

---
## Resumen Ejecutivo

Se realizó el procesamiento y enriquecimiento del dataset de encuestas de Fundación Teletón:

**Transformaciones realizadas:**
1. Conversión de tipos de datos óptimos (int8 para escalas, category para categóricas)
2. Creación de variables temporales (turno, día de semana, hora)
3. Cálculo de scores SERVQUAL por dimensión y total
4. Creación de variables discretas (NPS categoria, nivel satisfacción, etc.)
5. Enriquecimiento geográfico (región, población, PIB per cápita)

**Dataset final:** 274 registros x 38 columnas

**Archivos exportados:**
- `teleton_enriched.csv`
- `teleton_enriched.parquet`